# 텐서플로 기초

텐서플로 패키지를 임포트하여 아무런 에러가 나타나지 않으면 올바르게 설치된 것으로 보아도 됩니다.

In [1]:
import tensorflow as tf

/Users/rickypark/anaconda3/envs/hongdae-ml-study/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## 상수

텐서플로는 계산 그래프라고 부르는 자료 구조를 먼저 만들고, 그다음 이를 실행하여 실제 계산을 수행합니다. 따라서 그래프를 만드는 구성(construction) 단계에서는 아무런 값을 얻을 수 없습니다.

다음은 텐서플로의 기본 자료형인 상수(constant)를 하나 만듭니다. c를 출력하면 값 1 대신 텐서(Tensor) 타입의 객체를 출력합니다.

In [2]:
c = tf.constant(1)
c

<tf.Tensor 'Const:0' shape=() dtype=int32>

보통 딥러닝에서 __'텐서'__ 라고 하면 다차원 배열을 의미합니다. 이는 NumPy의 ndarray()와 거의 동급으로 생각할 수 있습니다. 하지만 사실 tf.Tensor는 다차원 배열이 아니고 그런 배열을 가리키는 포인터에 가깝습니다. 좀 더 기술적으로 설명하면 상수 c의 연산 노드('Const:0')를 가리킨다고 말할 수 있습니다.

두 개의 상수를 더하는 식을 만들어 보겠습니다.

In [3]:
d = tf.constant(2)
e = c + d
e

<tf.Tensor 'add:0' shape=() dtype=int32>

텐서 e는 덧셈 연산 'add:0'을 가리키고 있습니다. 이렇게 보면 e를 다차원 배열이라고 획일적으로 취급하기가 어렵습니다. 반면 c + d의 계산 결과를 얻기 위해서는 e를 사용하는 것이 맞으므로 흔히 텐서를 다차원 배열로 부릅니다. 보통 텐서, 연산, 연산 노드 등으로 다양하게 혼용하여 부릅니다.

그럼 이제 실제로 만들어진 그래프를 실행해 보겠습니다. 텐서플로 연산을 실행하기 위해서는 세션(Session) 객체를 만들어 run() 메소드를 호출해야 합니다.

In [4]:
sess = tf.Session()
sess.run(e)

3

기대했던 3이 출력되었습니다. 연산 e를 계산하기 위해 실행하면 이에 의존하는 c와 d도 계산되어 (그래프 구조의 잇점) 덧셈이 일어납니다. 하지만 c와 d는 여전히 그대로 텐서일 뿐 실제 값을 출력하지 않습니다.

In [5]:
c, d

(<tf.Tensor 'Const:0' shape=() dtype=int32>,
 <tf.Tensor 'Const_1:0' shape=() dtype=int32>)

e와 마찬가지로 c, d의 값을 얻으려면 sess.run() 메소드를 사용해야 합니다. 앞에서는 sess.run() 메소드에 하나의 연산을 넣었지만 여러개의 연산을 한꺼번에 넣을 수도 있습니다. 이 때에는 파이썬 리스트로 만들어 전달합니다. 반환되는 값도 역시 리스트입니다.

In [6]:
sess.run([c, d])

[1, 2]

이렇게 텐서플로는 계산 그래프의 구성(construction)와 실행(execution)이 나뉘어져 있는 것이 특징입니다. 앞서 만든 c, d, e 노드는 기본 계산 그래프에 추가됩니다. 기본 계산 그래프는 텐서플로 패키지를 임포트하면서 바로 사용할 수 있는 전역 그래프입니다. 

In [7]:
g = tf.get_default_graph()
g

get_operations() 메서드는 그래프에 있는 모든 연산을 반환합니다. 앞서 추가했던 세 개의 연산 노드가 있는 것을 확인할 수 있습니다.

In [8]:
g.get_operations()

[<tf.Operation 'Const' type=Const>,
 <tf.Operation 'Const_1' type=Const>,
 <tf.Operation 'add' type=Add>]

하지만 기본 그래프에 연습용 노드를 계속 추가하는 것은 복잡하고 버그를 발생시키기 쉽습니다. 따라서 기본 그래프 대신에 명시적으로 그래프를 만들어 예제를 진행하도록 하겠습니다. 다음과 같이 tf.Graph 객체를 만들고 파이썬의 with 컨텍스트로 원하는 그래프를 디폴트로 지정합니다.

In [9]:
g = tf.Graph()

with g.as_default() as g:
    c = tf.constant(1)

c

<tf.Tensor 'Const:0' shape=() dtype=int32>

조금 더 간단히 하려면 그래프를 생성하는 명령을 with 컨텍스트 안으로 집어 넣을 수 있습니다. 이전과 달리 새로운 그래프에 상수 연산이 추가되었으므로 텐서의 이름이 'Const_1'이 되지 않고 계속 'Const'로 유지되는 것을 볼 수 있습니다.

In [10]:
with tf.Graph().as_default() as g:
    c = tf.constant(3)

c

<tf.Tensor 'Const:0' shape=() dtype=int32>

세션은 기본 그래프의 연산을 실행하므로 앞서 만든 그래프를 명시적으로 지정해 줄 필요가 있습니다. 앞서와 마찬가지로 매번 세션 객체를 만들어 사용하기 보다는 with 컨텍스트 안에서 사용하는 것이 좋습니다.

In [11]:
with tf.Session(graph=g) as sess:
    print(sess.run(c))

3


일반 프로그래밍 언어에서의 의미와 같이 상수는 텐서플로가 계산 그래프를 실행하는 도중에 값이 변하지 않습니다.

상수는 하나의 스칼라 값 뿐만 아니라 다차원 배열도 가능하며 여러가지 데이터 타입(tf.int16, tf.int32, tf.int64, tf.float16, tf.float32, tf.float64)을 지정할 수 있습니다. 아무 값도 지정하지 않으면 넘겨진 값에 따라 tf.int32, tf.float32가 자동으로 할당됩니다. 만약 데이터 형을 명시적으로 지정하고 싶을 땐 dtype 매개변수를 사용합니다.

In [12]:
with tf.Graph().as_default() as g:
    c = tf.constant(1.0)

print(c)

with tf.Graph().as_default() as g:
    c = tf.constant(1.0, dtype=tf.float64)
    
print(c)

Tensor("Const:0", shape=(), dtype=float32)
Tensor("Const:0", shape=(), dtype=float64)


상수 배열에 정수와 실수가 섞여 있을 경우 자동으로 실수로 통일됩니다.

In [13]:
with tf.Graph().as_default() as g:
    c = tf.constant([[1., 2., 3.,], [4, 5, 6]])

with tf.Session(graph=g) as sess:
    print(sess.run(c))
c

[[1. 2. 3.]
 [4. 5. 6.]]


<tf.Tensor 'Const:0' shape=(2, 3) dtype=float32>

상수의 크기를 보면 2x3 행렬임을 알 수 있습니다. Scikit-Learn을 비롯하여 많은 라이브러리들이 행(row)에 샘플을 열(column)에 특성을 놓습니다. 하지만 그 반대의 경우도 간혹 있습니다. 여기에서는 일반적인 관례를 따라 행(row)을 따라 샘플을 나열하겠습니다.

텐서플로에서는 이를 2차원 텐서, 2x3 크기의 텐서 등으로 부릅니다.

사실 sess.run() 메서드에서 전달되는 반환값은 파이썬의 NumPy 패키지의 배열 객체입니다. NumPy는 파이썬의 대표적인 다차원 배열을 위한 패키지로 파이썬 과학 패키지들의 데이터 타입의 표준이 되어가고 있습니다. 그래서 Scikit-Learn과 TensorFlow에서도 기본 데이터 타입으로 NumPy를 사용합니다.

In [14]:
with tf.Session(graph=g) as sess:
    out = sess.run(c)
type(out)

numpy.ndarray

## 변수

상수는 계산 그래프를 실행하는 동안 값이 변하지 않으므로 뭔가 쓸모있는 것을 저장하는 용도로는 사용할 수 없습니다. 다른 범용 프로그래밍 언어와 마찬가지로 텐서플로에도 값을 저장하고 변경시킬 수 있는 변수가 있습니다.

tf.Variable에 전달한 첫 번째 값은 변수의 초기값입니다. 상수와 마찬가지로 dtype 매개변수를 사용하여 초기값의 자료형을 명시적으로 지정할 수 있습니다.

In [15]:
with tf.Graph().as_default() as g:
    v = tf.Variable([1., 2., 3.])
   
print(v)

with tf.Graph().as_default() as g:
    v = tf.Variable([1., 2., 3.], dtype=tf.float64)
    
print(v)

<tf.Variable 'Variable:0' shape=(3,) dtype=float32_ref>
<tf.Variable 'Variable:0' shape=(3,) dtype=float64_ref>


상수와는 달리 변수를 세션 객체로 바로 실행하면 에러가 발생합니다.

In [16]:
with tf.Session(graph=g) as sess:
    sess.run(v)

FailedPreconditionError: Attempting to use uninitialized value Variable
	 [[Node: _retval_Variable_0_0 = _Retval[T=DT_DOUBLE, index=0, _device="/job:localhost/replica:0/task:0/device:CPU:0"](Variable)]]

상수는 값이 고정되어 있어서 상수 연산('Const')에 직접 값이 포함되어 있습니다. 하지만 변수는 값이 변경되어야 하므로 연산과 실제 값이 나뉘어져 있습니다. 이 그래프에서 만들어진 연산 노드를 모두 출력해 보겠습니다.

In [17]:
g.get_operations()

[<tf.Operation 'Variable/initial_value' type=Const>,
 <tf.Operation 'Variable' type=VariableV2>,
 <tf.Operation 'Variable/Assign' type=Assign>,
 <tf.Operation 'Variable/read' type=Identity>]

변수 하나를 만들었는데 4개의 연산이 생겼습니다. 'Variable/initial_value'가 초기값을 가지고 있으며 'Varaible/Assign' 노드에 의해 'Variable'의 값이 할당 되고 'Variable/read' 연산으로 바뀐 값을 읽을 수 있습니다. 그럼 초기 값을 할당해 보겠습니다.

In [18]:
with tf.Session(graph=g) as sess:
    sess.run('Variable/Assign')
    print(sess.run(v))

[1. 2. 3.]


기대한 대로 값이 출력되었습니다. 하지만 변수가 많으면 이런 방식은 불편합니다. 대신 모든 변수의 초기와 연산을 한번에 모아서 실행시켜 주는 함수를 사용합니다.

In [19]:
with tf.Session(graph=g) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    print(sess.run(v))

[1. 2. 3.]


global_variables_initializer()는 변수의 초기와 연산을 모아 주는 것외엔 특별한 역할이 없으므로 보통 다음과 같이 줄여서 많이 사용합니다.

In [20]:
with tf.Session(graph=g) as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(v))

[1. 2. 3.]


고수준의 텐서플로 함수에서는 변수를 자동으로 만들어 주지만 저수준의 함수를 사용할 경우에는 변수를 직접 만들어 줍니다. 어떤 경우에도 전체 변수를 초기화하는 부분은 동일합니다.

변수와 관련된 흔한 실수 중에 하나가 다음과 같이 같은 변수에 덧셈 연산을 적용하는 것입니다.

In [21]:
with tf.Graph().as_default() as g:
    v = tf.Variable([1, 2, 3])

print(v)

v = v + 1

print(v)

<tf.Variable 'Variable:0' shape=(3,) dtype=int32_ref>
Tensor("add:0", shape=(3,), dtype=int32)


v는 tf.Variable의 객체였지만 덧셈 연산에 할당하는 순간 연산의 출력을 가리키는 텐서가 되었습니다. 그래서 v를 여러번 실행해도 덧셈 연산은 그대로 1을 출력합니다. 즉 덧셈의 결과가 원래 변수 v에 할당되지 않습니다.

In [22]:
with tf.Session(graph=g) as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(v))
    print(sess.run(v))

[2 3 4]
[2 3 4]


이런 문제를 피하려면 텐서플로에서 제공하는 할당 연산자를 사용해야 합니다.

In [23]:
with tf.Graph().as_default() as g:
    v = tf.Variable([1, 2, 3])
    add = v + 1
    asn = tf.assign(v, add)
v, add, asn

(<tf.Variable 'Variable:0' shape=(3,) dtype=int32_ref>,
 <tf.Tensor 'add:0' shape=(3,) dtype=int32>,
 <tf.Tensor 'Assign:0' shape=(3,) dtype=int32_ref>)

v가 변경되지 않고 여전히 변수를 가리키고 있습니다. 또한 v + 1은 덧셈 연산의 출력을 가리키는 텐서가 되었고 tf.assign()은 할당 연산의 출력을 가리키는 텐서가 되었습니다. 만약 asn을 실행시키면 의존성을 따라 자동으로 add, v+1이 계산될 것입니다.

In [24]:
with tf.Session(graph=g) as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(v))
    print(sess.run(asn))
    print(sess.run(asn))

[1 2 3]
[2 3 4]
[3 4 5]


결과에서 눈치챌 수 있듯이 1을 더했는데 배열 전체에 더해졌습니다. 이를 브로드캐스팅이라고 하며 파이썬의 과학 패키지에서는 일반적으로 기대되는 기능입니다.

보통은 덧셈 연산 add가 간단하므로 따로 만들지 않고 assign 함수에 직접 덧셈 식을 주입합니다. 하지만 덧셈 연산이 만들어지고 asn 계산에 사용되는 방식은 동일합니다.

In [25]:
with tf.Graph().as_default() as g:
    v = tf.Variable([1, 2, 3])
    asn = tf.assign(v, v + 1)

with tf.Session(graph=g) as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(asn))
    print(sess.run(asn))

[2 3 4]
[3 4 5]


## 행렬

신경망의 기본 연산은 행렬 연산입니다. 다음으로 넘어가기 전에 잠시 행렬 연산에 대해 알아 보도록 하겠습니다.

먼저 행렬에 표현할 때 다음과 같이 두 개의 행과 3개의 열이 있을 때 이를 [2x3] 행렬이라고 표현합니다.

$$\left\lgroup\matrix{ 1 & 2 & 3 \\ 4 & 5 & 6 }\right\rgroup$$

이전과 같이 1을 더하면 텐서플로의 브로드캐스팅 덕에 모든 원소에 1이 더해집니다.

$$\left\lgroup\matrix{ 1 & 2 & 3 \\ 4 & 5 & 6 }\right\rgroup + 1 = \left\lgroup\matrix{ 2 & 3 & 4 \\ 5 & 6 & 7 }\right\rgroup$$

이는 마치 모든 원소가 1인 [2x3] 행렬을 더하는 것과 같지만 훨씬 빠르게 작동됩니다.

In [26]:
with tf.Graph().as_default() as g:
    v = tf.Variable([[1, 2, 3], [4, 5, 6]])
    
with tf.Session(graph=g) as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(v + 1))
v

[[2 3 4]
 [5 6 7]]


<tf.Variable 'Variable:0' shape=(2, 3) dtype=int32_ref>

신경망에서 가장 핵심이 되는 행렬 연산은 점곱(dot product)입니다. 아래와 같이 첫 번째 행렬의 행이 두 번째 행렬의 열과 원소별로 곱해져서 덧셈이 되어 결과 행렬의 하나의 원소가 됩니다. 점곱 연산은 첫 번째 행렬의 위에서 아래로, 두 번째 행렬의 왼쪽에서 오른쪽으로 진행됩니다.

$$\left\lgroup\matrix{ 1 & 2 & 3 \\ 4 & 5 & 6 }\right\rgroup + \left\lgroup\matrix{ 1 & 2 \\ 3 & 4 \\ 5 & 6 }\right\rgroup = \left\lgroup\matrix{ 22 & 28 \\ 49 & 64 }\right\rgroup$$

첫 번째 행렬의 행과 두 번째 행렬의 열이 곱해지다 보니 첫 번째 행렬의 열과 두 번째 행렬의 행의 크기가 같아야 합니다. 그리고 첫 번째 행렬의 행과 두 번째 행렬의 열이 결과 행렬의 행과 열의크기가 됩니다.

$$ [2\times3]\cdot[3\times2] = [2\times2]$$

In [27]:
with tf.Graph().as_default() as g:
    v1 = tf.Variable([[1, 2, 3], [4, 5, 6]])
    v2 = tf.Variable([[1, 2], [3, 4], [5, 6]])
    
with tf.Session(graph=g) as sess:
    sess.run(tf.global_variables_initializer())
    m = tf.matmul(v1, v2)
    print(sess.run(m))

[[22 28]
 [49 64]]


행렬의 순서를 바꾸어서 점곱하면 $[3\times2]\cdot[2\times3]=[3\times3]$의 결과를 얻게 됩니다.

In [28]:
with tf.Session(graph=g) as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(tf.matmul(v2, v1)))

[[ 9 12 15]
 [19 26 33]
 [29 40 51]]


## 플레이스홀더

신경망은 데이터로부터 의미있는 패턴을 학습하는 모델 기반 머신러닝 알고리즘입니다. 즉 모델에 있는 어떤 파라미터의 값을 데이터로부터 얻어내어 저장해야 합니다. 이런 모델의 파라미터를 모델 파라미터라고 부르며 모델 파라미터를 얻어 내는 과정을 학습이라고 합니다. 상수와 달리 변수는 값을 바꿀 수 있기 때문에 모델 파라미터를 저장하는데 유용합니다.

훈련 데이터는 근본적으로 변하는 것이 아니기 때문에 상수나 변수로 저장하여 계산 그래프를 만들 수도 있습니다. 그러나 이렇게 되면 그래프의 크기(용량)가 매우 커져서 CPU, GPU에서 계산을 수행할 때 비효율적이며 어떤 경우에는 그래프로 만들기에는 훈련 데이터가 너무 큽니다. 신경망은 대체적으로 대용량의 데이터를 다루기 때문에 이런 경우가 대부분입니다.

그래서 계산 그래프를 만들고 난 후에 외부에서 데이터를 따로 전달해 주어야 하는 통로가 필요하게 됩니다. 특히 훈련 데이터를 조금씩 나누어 신경망 모델을 훈련시킬 때는 더욱 그렇습니다. 이렇게 데이터의 구조를 만들고 실행시에 데이터를 받기 위한 것이 플레이스홀더(placeholder)입니다. 간단한 플레이스홀더를 하나 만들어 보겠습니다.

In [29]:
with tf.Graph().as_default() as g:
    h = tf.placeholder(tf.int32, shape=(2, 3))

h

<tf.Tensor 'Placeholder:0' shape=(2, 3) dtype=int32>

플레이스홀더의 첫 번째 매개변수는 데이터형이고 두 번째 매개변수는 자료의 크기입니다. 여기에서는 [2, 3]로 2차원 행렬(또는 텐서)를 지정했습니다. 즉 [2x3] 크기의 다차원 배열이 저장될 곳이라는 의미입니다. 두 번째 매개변수의 키워드 shape은 생략해도 되지만 가능하면 명시적으로 써 주는 것이 코드를 읽을 때 좋습니다.

h도 텐서이고 연산은 Placeholder입니다.

In [30]:
h.op, h.op.name

(<tf.Operation 'Placeholder' type=Placeholder>, 'Placeholder')

플레이스홀더 h를 세션을 만들어 실행해 보겠습니다. 변수가 아니므로 별도로 초기화를 해 주어야할 필요는 없습니다.

In [31]:
with tf.Session(graph=g) as sess:
    print(sess.run(h))

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder' with dtype int32 and shape [2,3]
	 [[Node: Placeholder = Placeholder[dtype=DT_INT32, shape=[2,3], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Placeholder', defined at:
  File "/Users/rickypark/anaconda3/envs/hongdae-ml-study/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/rickypark/anaconda3/envs/hongdae-ml-study/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/rickypark/anaconda3/envs/hongdae-ml-study/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/rickypark/anaconda3/envs/hongdae-ml-study/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/rickypark/anaconda3/envs/hongdae-ml-study/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 478, in 

플레이스홀더는 값을 담는 그릇으로 볼 수 있기 때문에 실행해서 얻을 수 있는 텐서가 없습니다. 플레이스홀더를 실행하려면 반드시 담을 값을 함께 주입해 주어야 합니다. 이 때 사용되는 매개변수가 sess.run() 메서드의 feed_dict 입니다. feed_dict 는 파이썬의 딕셔너리 형태로 플레이스홀더의 변수가 키가 되고 주입할 데이터를 값으로 지정합니다. 값으로 넣을 수 있는 것은 파이썬 데이터형이나 텐서, 넘파이 배열 등입니다.

In [32]:
with tf.Session(graph=g) as sess:
    print(sess.run(h, feed_dict={h: [ [1, 2, 3], [4, 5, 6] ]}))

[[1 2 3]
 [4 5 6]]


플레이스홀더는 미리 정해진 크기와 다른 크기의 값이 전해지면 에러를 냅니다. $[3\times1]$ 크기의 배열을 넣어 보겠습니다.

In [33]:
with tf.Session(graph=g) as sess:
    print(sess.run(h, feed_dict={h: [ [1, 2, 3] ]}))

ValueError: Cannot feed value of shape (1, 3) for Tensor 'Placeholder:0', which has shape '(2, 3)'

대신 플레이스홀더는 실행할 때마다 매번 다른 값을 넣을 수 있습니다. 그렇기 때문에 신경망 알고리즘을 훈련시킬 때 훈련 데이터를 주입할 수 있는 주요 통로가 됩니다.

In [34]:
with tf.Session(graph=g) as sess:
    h_out = sess.run(h, feed_dict={h: [ [1, 2, 3], [4, 5, 6] ]})
    print(h_out)
    h_out = sess.run(h, feed_dict={h: [ [7, 8, 9], [10, 11, 12] ]})
    print(h_out)

[[1 2 3]
 [4 5 6]]
[[ 7  8  9]
 [10 11 12]]


한가지 재미있는 기능은 플레이스홀더에 크기를 None으로 지정하여 임의의 크기의 플레이스홀더를 만들 수 있다는 점입니다. 이런 기능이 유용한 때는 훈련 데이터의 개수가 정확히 결정되지 않았을 때 유리합니다. 앞에서 만든 플레이스홀더 h에 첫 번째 차원을 None으로 만들어 보겠습니다.

In [35]:
with tf.Graph().as_default() as g:
    h = tf.placeholder(tf.int32, shape=(None, 3))
    
with tf.Session(graph=g) as sess:
    h_out = sess.run(h, feed_dict={h: [ [1, 2, 3], [4, 5, 6] ]})
    print(h_out)
    h_out = sess.run(h, feed_dict={h: [ [1, 2, 3] ]})
    print(h_out)

[[1 2 3]
 [4 5 6]]
[[1 2 3]]


앞의 예와는 달리 입력값을 $[2\times3]$와 $[1\times3]$로 첫 번째 차원을 달리하여 데이터를 주입했는 데에도 에러가 발생하지 않았습니다.

플레이스홀더가 사용되는 또 하나의 요긴한 사용처가 하이퍼파라미터입니다. 하이퍼파라미터는 모델 파라미터와는 달리 훈련 데이터로부터 학습되지 않고 우리가 지정해 주어야 하는 값입니다. 그런데 이 값이 훈련할 때와 테스트할 때 서로 값이 달라야 할 경우가 있습니다. 만약 변수나 상수로 이런 하이퍼파라미터를 지정했다면 그래프를 다시 만들어야 하기 때문에 낭패입니다. 플레이스홀더를 사용하면 훈련할 때와 테스트할 때 각기 다른 하이퍼파라미터를 주입할 수 있습니다.

플레이스홀더에는 기본값을 지정할 수 있는 tf.placeholder_with_default() 함수가 있습니다. 이 함수는 그래프에 하이퍼파라미터와 같이 상황에 따라 달라질 수 있는 값의 기본값을 지정하는 데 유용합니다. 이 함수의 첫 번째 매개변수는 데이터 타입이 아니고 기본값입니다.

In [36]:
with tf.Graph().as_default() as g:
    h = tf.placeholder_with_default([[1, 2, 3]], shape=(None, 3))
    
with tf.Session(graph=g) as sess:
    h_out = sess.run(h, feed_dict={h: [ [1, 2, 3], [4, 5, 6] ]})
    print(h_out)
    h_out = sess.run(h) # 값을 주입하지 않으므로 기본값이 들어갑니다.
    print(h_out)

[[1 2 3]
 [4 5 6]]
[[1 2 3]]
